In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

rating_df = pd.read_csv("/kaggle/input/anime-recommendations-database/rating.csv")
anime_df = pd.read_csv("/kaggle/input/anime-recommendations-database/anime.csv")
print("read")

In [ ]:
anime_df.head()

In [ ]:
anime_df.info()

Lets remove columns that we are not going to use right now. Maybe in the future we can use them to better predict what to recommend for our costumers.

In [ ]:
#We don't need 'rating' column, we are going to use the ratings from the ratings dataframe.
#The number of members, the type and the numer of the episodes in the anime maybe be a good estimador,
#but now we are going to focus only on the anime's genre.

animes_with_genres_df = anime_df.copy()

animes_with_genres_df.drop(['rating', 'members', 'type', 'episodes'], axis=1, inplace=True)
animes_with_genres_df.head()

In [ ]:
animes_with_genres_df.isna().sum()

Temos alguns valores desconhecidos (NaN) no dataset, nas colunas genre, type e rating, aparentemente são poucos, então podemos lidar mais tranquilamente com eles, talvez mesmo remove-los do dataset.

In [ ]:
animes_with_genres_df.dropna(inplace=True)
animes_with_genres_df.info()

A coluna "genre" possui valores em string com uma lista das categorias na qual aquele anime se encaixa. Dados assim não ajudam nossos algoritmos de Machine Learning, então a primeira etapa será criar uma coluna no dataset para cada gênero de anime, e colocar um valor de 1 para se o anime pertencer àquele gênero, e 0 para se ele não pertencer.

In [ ]:
for index, row in animes_with_genres_df.iterrows():
    for genre in row.genre.split(", "):
        animes_with_genres_df.at[index, genre] = 1
        
animes_with_genres_df.fillna(0, inplace=True)
animes_with_genres_df.head()

Agora temos uma coluna para representar cada gênero possível do anime, e o valor 1 para se o anime pertencer àquele gênero e 0 para se ele não pertencer.

In [ ]:
rating_df.head()

In [ ]:
rating_df.info()

Agora, vamos criar um content-based recommendation system. O foco dele será recomendar animes novos para um possível consumidor, baseado em notas que o cliente deu para outros títulos. Levaremos em conta os gêneros dos animes para recomendar similares a ele.

In [ ]:
animes_with_genres_df.tail()

In [ ]:
#Vamos criar um consumidor ficticio, com alguns animes na lista dele e algumas notas.
user_test = [
    {'name': 'Kimi no Na wa.', 'rating': 10},
    {'name': 'Fullmetal Alchemist: Brotherhood', 'rating': 10},
    {'name': 'Naruto', 'rating': 8},
    {'name': 'Under World', 'rating': 3},
    {'name': 'Yasuji no Pornorama: Yacchimae!!', 'rating': 4},
]

user_test_df = pd.DataFrame(user_test)
user_test_df.head()

In [ ]:
# Vamos unir a lista com os dados do titulo que o usuario assistiu com o respectivo titulo no dataframe
#dos animes.
user_test_df = pd.merge(animes_with_genres_df, user_test_df, on ="name")

# Vamos separar os dados agora para um dataframe que contem apenas o id do anime, o nome e a 
#nota que o cliente deu
# E vamos criar uma lista com os mesmos dados (exceto rating) e os restante dos dados
user_test_animes = user_test_df.drop(['rating'], axis=1)
user_test_df = user_test_df[['anime_id', 'name', 'rating']]

print(user_test_animes)
print(user_test_df)

In [ ]:
# Agora vamos pegar apenas os gêneros da lista do cliente, mas antes vamos reiniciar os index para
#evitar problemas futuros
user_test_animes = user_test_animes.reset_index(drop=True)
user_test_genres = user_test_animes.drop(['anime_id', 'name', 'genre'], axis=1)
user_test_genres

Agora, iremos multiplicar a nota dada pelo cliente pelos gêneros dos títulos, assim transformaremos cada gênero em um peso para a avaliação final. Como é um produto escalar, podemos usar a função **dot** do pandas

In [ ]:
user_profile = user_test_genres.transpose().dot(user_test_df['rating'])
user_profile

Esses pesos são conhecidos como "Perfil do usuário", baseado neles podemos recomendar novos títulos para o cliente

Agora vamos pegar a lista de gêneros do dataframe original

In [ ]:
genre_table = animes_with_genres_df.set_index(animes_with_genres_df['anime_id'])

genre_table.drop(['anime_id', 'name', 'genre'], axis=1, inplace=True)
genre_table.head()

In [ ]:
genre_table.shape

Agora vamos pegar a nossa lista de gêneros, nossos pesos e calcular a média ponderada de cada título para recomendar os que mais batem com o gosto do cliente.

In [ ]:
recommendations_df = ((genre_table * user_profile).sum(axis=1)) / (user_profile.sum())
recommendations_df.head()

In [ ]:
# Vamos ordená-los em ordem decrescente
recommendations_df = recommendations_df.sort_values(ascending=False)
recommendations_df.head()

In [ ]:
anime_df.loc[anime_df['anime_id'].isin(recommendations_df.head(10).keys())]